In [1]:
install.packages("dummies")
install.packages("fastDummies")
install.packages('tidyverse')

Updating HTML index of packages in '.Library'
Making 'packages.html' ... done
Updating HTML index of packages in '.Library'
Making 'packages.html' ... done
Updating HTML index of packages in '.Library'
Making 'packages.html' ... done


In [2]:
library(readr)
library(dplyr)
library(dummies)
library(fastDummies)
library(tidyverse)


Attaching package: ‘dplyr’

The following objects are masked from ‘package:stats’:

    filter, lag

The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union

dummies-1.5.6 provided by Decision Patterns

── Attaching packages ─────────────────────────────────────── tidyverse 1.2.1 ──
✔ ggplot2 3.0.0     ✔ purrr   0.2.5
✔ tibble  1.4.2     ✔ stringr 1.3.1
✔ tidyr   0.8.1     ✔ forcats 0.3.0
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()


In [3]:
historical_transactions <- read_csv("/Users/eve7947/Downloads/elo-merchant-category-recommendation/historical_transactions.csv")
merchants <- read_csv("/Users/eve7947/Downloads/elo-merchant-category-recommendation/merchants.csv")
new_merchant_transactions <- read_csv("/Users/eve7947/Downloads/elo-merchant-category-recommendation/new_merchant_transactions.csv")
sample_submission <- read_csv("/Users/eve7947/Downloads/elo-merchant-category-recommendation/sample_submission.csv")
test <- read_csv("/Users/eve7947/Downloads/elo-merchant-category-recommendation/test.csv")
train <- read_csv("/Users/eve7947/Downloads/elo-merchant-category-recommendation/train.csv")

Parsed with column specification:
cols(
  authorized_flag = col_character(),
  card_id = col_character(),
  city_id = col_integer(),
  category_1 = col_character(),
  installments = col_integer(),
  category_3 = col_character(),
  merchant_category_id = col_integer(),
  merchant_id = col_character(),
  month_lag = col_integer(),
  purchase_amount = col_double(),
  purchase_date = col_datetime(format = ""),
  category_2 = col_double(),
  state_id = col_integer(),
  subsector_id = col_integer()
)
Parsed with column specification:
cols(
  .default = col_double(),
  merchant_id = col_character(),
  merchant_group_id = col_integer(),
  merchant_category_id = col_integer(),
  subsector_id = col_integer(),
  category_1 = col_character(),
  most_recent_sales_range = col_character(),
  most_recent_purchases_range = col_character(),
  active_months_lag3 = col_integer(),
  active_months_lag6 = col_integer(),
  active_months_lag12 = col_integer(),
  category_4 = col_character(),
  city_id = col_in

### Training Data

#### Check Data Quality

In [4]:
summary(train)

 first_active_month   card_id            feature_1       feature_2    
 Length:201917      Length:201917      Min.   :1.000   Min.   :1.000  
 Class :character   Class :character   1st Qu.:2.000   1st Qu.:1.000  
 Mode  :character   Mode  :character   Median :3.000   Median :2.000  
                                       Mean   :3.105   Mean   :1.745  
                                       3rd Qu.:4.000   3rd Qu.:2.000  
                                       Max.   :5.000   Max.   :3.000  
   feature_3          target         
 Min.   :0.0000   Min.   :-33.21928  
 1st Qu.:0.0000   1st Qu.: -0.88311  
 Median :1.0000   Median : -0.02344  
 Mean   :0.5656   Mean   : -0.39364  
 3rd Qu.:1.0000   3rd Qu.:  0.76545  
 Max.   :1.0000   Max.   : 17.96507  

In [5]:
dim(train)

[1] 201917      6

In [6]:
sum(is.na(train))

[1] 0

In [9]:
glimpse(train)

Observations: 201,917
Variables: 6
$ first_active_month <chr> "2017-06", "2017-01", "2016-08", "2017-09", "201...
$ card_id            <chr> "C_ID_92a2005557", "C_ID_3d0044924f", "C_ID_d639...
$ feature_1          <int> 5, 4, 2, 4, 1, 4, 3, 3, 2, 2, 5, 2, 5, 2, 3, 2, ...
$ feature_2          <int> 2, 1, 2, 3, 3, 2, 2, 2, 1, 2, 2, 2, 2, 1, 2, 1, ...
$ feature_3          <int> 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, ...
$ target             <dbl> -0.82028260, 0.39291325, 0.68805599, 0.14249520,...


In [11]:
library(lubridate)


Attaching package: ‘lubridate’

The following object is masked from ‘package:base’:

    date



In [20]:
 train %>% 
         first_active_month = ymd(train$first_active_month, truncated = 1) %>% 
  ggplot(aes(x = first_active_month, fill = set)) +
  geom_bar() +
  theme_minimal()

ERROR: Error: `data` must be a data frame, or other object coercible by `fortify()`, not an S3 object with class Date


#### Feature: Target

In [78]:
#boxplot(train$target)

In [86]:
sum(train$target < -30)

[1] 2207

In [87]:
train %>% select(card_id, target) %>% filter(target < -30) %>% head()

card_id,target
C_ID_8186f3fcc1,-33.21928
C_ID_b9379a30ea,-33.21928
C_ID_e9120f535c,-33.21928
C_ID_65715cb80d,-33.21928
C_ID_ae77d244b6,-33.21928
C_ID_c4262c902e,-33.21928


In [88]:
train_nooutliers <- train[train$target > -30,]
# Remove Outliers with Target < -30

In [92]:
Output <- select(train_nooutliers, card_id, target)

In [93]:
head(Output)

card_id,target
C_ID_92a2005557,-0.8202826
C_ID_3d0044924f,0.3929132
C_ID_d639edf6cd,0.6880560
C_ID_186d6a6901,0.1424952
C_ID_cdbd2c0db2,-0.1597492
C_ID_0894217f2f,0.8715853


In [94]:
Input <- select(train_nooutliers, -target)

#### Features1-3

In [95]:
#hist(train$feature_1) #do for feature 1-3 and make it one picture

In [99]:
Input <-fastDummies::dummy_cols(Input, select_columns = c("feature_1","feature_2","feature_3")) # change to dummy column
Input <- select(Input,-feature_1,-feature_2,-feature_3)

In [100]:
head(Input)

first_active_month,card_id,feature_1_5,feature_1_4,feature_1_2,feature_1_1,feature_1_3,feature_2_2,feature_2_1,feature_2_3,feature_3_1,feature_3_0
2017-06,C_ID_92a2005557,1,0,0,0,0,1,0,0,1,0
2017-01,C_ID_3d0044924f,0,1,0,0,0,0,1,0,0,1
2016-08,C_ID_d639edf6cd,0,0,1,0,0,1,0,0,0,1
2017-09,C_ID_186d6a6901,0,1,0,0,0,0,0,1,0,1
2017-11,C_ID_cdbd2c0db2,0,0,0,1,0,0,0,1,0,1
2016-09,C_ID_0894217f2f,0,1,0,0,0,1,0,0,0,1
